In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import ipywidgets as widgets # provides interactive functionality
from IPython.display import HTML, display, clear_output # display and html functionality
import pandas as pd
import csv # csv import functionality
import warnings # needed for suppression of unnecessary warnings
import base64 # functionality for encoding binary data to ASCII characters and decoding back to binary data
import fit_models as fits
import plot_fits as plot_fits
import hospital_needs as needs

warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'svg' # configure to display scalable vector graphics

In [12]:
testing = 0

if testing == 1:
    seir_fits_df = pd.read_csv('SEIR-SD_States_Update.txt', sep='\t')
else:
    try:
        seir_fits_df = pd.read_csv('https://raw.githubusercontent.com/Rush-Quality-Analytics/SupplyDemand/master/notebooks/SEIR-SD_States_Update.txt', sep='\t')
    except:
        seir_fits_df = pd.read_csv('SEIR-SD_States_Update.txt', sep='\t')

if testing == 1:
    statepops = pd.read_csv('StatePops.csv')
else:
    try: 
        path = 'https://raw.githubusercontent.com/Rush-Quality-Analytics/SupplyDemand/master/notebooks/StatePops.csv'
        statepops = pd.read_csv('path')

    except:
        statepops = pd.read_csv('StatePops.csv')

if testing == 1:
    ap_df = pd.read_csv('COVID-CASES-DF.txt', sep='\t') 
    ap_df = ap_df[ap_df['Country/Region'] == 'US']
    ap_df = ap_df[ap_df['Province/State'] != 'US']
    ap_df = ap_df[ap_df['Province/State'] != 'American Samoa']
    ap_df = ap_df[ap_df['Province/State'] != 'Northern Mariana Islands']
    ap_df.drop(columns=['Unnamed: 0'], inplace=True)
    
else:
    try: 
        ap_df = pd.read_csv('https://raw.githubusercontent.com/Rush-Quality-Analytics/SupplyDemand/master/notebooks/COVID-CASES-DF.txt', sep='\t') 
        ap_df = ap_df[ap_df['Country/Region'] == 'US']
        ap_df = ap_df[ap_df['Province/State'] != 'US']
        ap_df = ap_df[ap_df['Province/State'] != 'American Samoa']
        ap_df = ap_df[ap_df['Province/State'] != 'Northern Mariana Islands']
        ap_df.drop(columns=['Unnamed: 0'], inplace=True)
    except:
        ap_df = pd.read_csv('COVID-CASES-DF.txt', sep='\t') 
        ap_df = ap_df[ap_df['Country/Region'] == 'US']
        ap_df = ap_df[ap_df['Province/State'] != 'US']
        ap_df = ap_df[ap_df['Province/State'] != 'American Samoa']
        ap_df = ap_df[ap_df['Province/State'] != 'Northern Mariana Islands']
        ap_df.drop(columns=['Unnamed: 0'], inplace=True)



    
global app2    
app0 = fits.App_GetFits.fits(ap_df, statepops, seir_fits_df)
grid0 = widgets.GridspecLayout(1, 1)
grid0[0, 0] = app0.container
app_contents0 = [grid0]
app00 = widgets.VBox(app_contents0, layout=widgets.Layout(align_items='center'))
display(app00)



b1 = widgets.Button(description='Click to update plots and tables. Do this after running a different model and/or choosing a different state or territory.',
    disabled=False, button_style='info', tooltip='Click me', layout={'width': '1000px'}, icon='')
box_layout = widgets.Layout(display='flex',flex_flow='column',align_items='center',width='100%')
box1 = widgets.HBox(children=[b1],layout=box_layout)
display(box1)
app1 = plot_fits.App_PlotFits.generate_plot(app0._model_fits_df)

box_layout = widgets.Layout(display='flex',flex_flow='column',align_items='center',width='100%')

out2 = widgets.Output()
app2 = needs.App_GetNeeds.from_url(app0.ForecastDays, app0._model_fits_df, app0.new_cases)

with out2:    
    grid2 = widgets.GridspecLayout(1, 1)
    grid2[0, 0] = app2.container
    app_contents2 = [grid2]
    app22 = widgets.VBox(app_contents2, layout=widgets.Layout(align_items='center'))
    display(app22)
display(out2)


def on_button_clicked1(b1):
    global app2, app0
    def update_out2(out2):
        global app2, app0
        with out2:
            app2 = needs.App_GetNeeds.from_url(app0.ForecastDays, app0._model_fits_df, app0.new_cases)
            grid2 = widgets.GridspecLayout(1, 1)
            grid2[0, 0] = app2.container
            app_contents2 = [grid2]
            app22 = widgets.VBox(app_contents2, layout=widgets.Layout(align_items='center'))
            clear_output(wait=True)
            display(app22)
    update_out2(out2)
    
b1.on_click(on_button_clicked1)


Output()